In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import _pickle as cPickle

In [2]:
def unpickle(filename):
    fo = open(filename, 'rb')
    fdict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return fdict

In [3]:
data_dict = {}
for i in range(1, 6):
    data_dict['data_batch_' + str(i)] = unpickle('cucumber_data/p1/data_batch_' + str(i))
data_dict['test_batch'] = unpickle('cucumber_data/p1/test_batch')

In [4]:
def get_images(data, labels):
    num_images = data.shape[0] // 3
    images = np.zeros((num_images, 32, 32, 3))
    labels_arr = np.zeros((num_images, 9))
    for i in range(num_images):
        images[i] = np.vstack((data[3*i], data[3*i+1], data[3*i+2])).reshape(3, 32, 32).transpose(1,2,0) / 255
        labels_arr[i][labels[i]] = 1
    return (images, labels_arr)

In [5]:
images_dict = {}
labels_dict = {}
for data_batch in data_dict:
    images_dict[data_batch], labels_dict[data_batch] = get_images(data_dict[data_batch]['data'], data_dict[data_batch]['labels'])

In [6]:
train_images = np.vstack((images_dict['data_batch_1'], 
                          images_dict['data_batch_2'], 
                          images_dict['data_batch_3'], 
                          images_dict['data_batch_4']))

train_labels = np.vstack((labels_dict['data_batch_1'], 
                          labels_dict['data_batch_2'], 
                          labels_dict['data_batch_3'], 
                          labels_dict['data_batch_4']))

In [7]:
def jiggling_image(image):
    #noise = np.random.rand(image.shape[0], image.shape[1], image.shape[2])
    noise = np.random.normal(0, 0.01, image.shape)
    new_image = image + noise
    new_image = np.minimum(1, new_image)
    new_image = np.maximum(0, new_image)
    return new_image

def jiggling_images(image, label, new_num=3):
    new_images = np.zeros((new_num, image.shape[0], image.shape[1], image.shape[2]))
    #new_labels = np.ones((new_num, 1)) * int(label)
    new_labels = np.tile(label, new_num).reshape((new_num, 9))
    for i in range(new_num):
        new_images[i] = jiggling_image(image)
    return (new_images, new_labels)
        

len_train_images = len(train_images)

for i in range(len_train_images):
    new_images, new_labels = jiggling_images(train_images[i], train_labels[i])
    train_images = np.vstack((train_images, new_images))
    train_labels = np.vstack((train_labels, new_labels))
print(train_images.shape)

print(train_labels.shape)

In [10]:
validation_images = images_dict['data_batch_5']
validation_labels = labels_dict['data_batch_5']

test_images = images_dict['test_batch']
test_labels = labels_dict['test_batch']

In [11]:
def get_batch(images, batch_size):
    images_ind = list(range(len(images)))
    random.shuffle(images_ind)
    return images[images_ind[:batch_size]]

In [12]:
def leaky_relu(x, alpha=0.01):
    return tf.maximum(alpha*x, x)

## Convolutional Neural Network

In [13]:
import math
input_size = 32*32*3
filter_conv1 = 5
filter_conv2 = 10
#neuron_1 = 4*4*filter_conv2
neuron_1 = 8*8*filter_conv2
neuron_2 = neuron_1 // 8
W_stddev = 1

learning_rate = 1e-4
num_train_examples = train_images.shape[0]
sqrt_num_train_examples = math.sqrt(num_train_examples)
iters = 5*num_train_examples
batch_size = 200
num_subjects = 9
keep_prob = tf.placeholder(tf.float32)

In [14]:
x = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name='images')
y_correct = tf.placeholder(shape=[None, num_subjects], dtype=tf.float32, name='correct_output')

# first convolution
W_conv1 = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, filter_conv1], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.zeros([filter_conv1]), name="b_conv1")
h_conv1 = tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1   # (None, 32, 32, filter_conv1)
#h_conv1_relu = tf.nn.relu(h_conv1)
h_conv1_relu = leaky_relu(h_conv1)

# first max pool
h_pool_1 = tf.nn.max_pool(h_conv1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 16, 16, filter_conv1)

# second convolution
W_conv2 = tf.Variable(tf.truncated_normal(shape=[3, 3, filter_conv1, filter_conv2], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.zeros([filter_conv2]), name="b_conv2")
h_conv2 = tf.nn.conv2d(h_pool_1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2   # (None, 16, 16, filter_conv2)
#h_conv2_relu = tf.nn.relu(h_conv2)
h_conv2_relu = leaky_relu(h_conv2)


# second max pool
h_pool_2 = tf.nn.max_pool(h_conv2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 8, 8, filter_conv2)


In [15]:
# flatten
fc_input = tf.reshape(h_pool_2, [-1, neuron_1])

# Fully Connected
W_1 = tf.Variable(tf.truncated_normal([neuron_1, neuron_2], stddev=W_stddev) / sqrt_num_train_examples, name="W_1")
b_1 = tf.Variable(0.0, [neuron_2], name="b_1")
a_1 = tf.nn.relu(tf.matmul(fc_input, W_1) + b_1)

keep_prob = tf.placeholder(tf.float32)
a_1_drop = tf.nn.dropout(a_1, keep_prob)

W_2 = tf.Variable(tf.truncated_normal([neuron_2, num_subjects], stddev=W_stddev), name="W_2")
b_2 = tf.Variable(0.0, [num_subjects], name="b_2")
y = tf.matmul(a_1_drop, W_2) + b_2

In [16]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_correct, logits=y))

# Optimization
train_step = tf.train.AdamOptimizer(3e-4).minimize(cross_entropy)

# Evaluate model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

validation_feed_dict = {x: validation_images, y_correct: validation_labels, keep_prob: 1.0 }
test_feed_dict = {x: test_images, y_correct: test_labels, keep_prob: 1.0}
for i in range(iters):
    train_indices = random.sample(range(num_train_examples), batch_size)
    train_feed_dict = {x: train_images[train_indices], y_correct: train_labels[train_indices], keep_prob:0.4 }

    if i%2000 == 0:
        train_accuracy = accuracy.eval(feed_dict=train_feed_dict)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        print("Accuracy validation: {}".format(accuracy.eval(feed_dict = validation_feed_dict)))
    _ = sess.run(train_step, feed_dict = train_feed_dict)

print("DONE. Test accuracy: {}".format(accuracy.eval(feed_dict = test_feed_dict)))

step 0, training accuracy 0.125
Accuracy validation: 0.12121213972568512
step 2000, training accuracy 0.535
Accuracy validation: 0.6888889074325562
step 4000, training accuracy 0.585
Accuracy validation: 0.7515152096748352
step 6000, training accuracy 0.61
Accuracy validation: 0.7757576107978821
step 8000, training accuracy 0.615
Accuracy validation: 0.7717171907424927
step 10000, training accuracy 0.695
Accuracy validation: 0.7717171907424927
step 12000, training accuracy 0.69
Accuracy validation: 0.7717171907424927
step 14000, training accuracy 0.665
Accuracy validation: 0.7959597110748291
step 16000, training accuracy 0.69
Accuracy validation: 0.7959596514701843
step 18000, training accuracy 0.745
Accuracy validation: 0.7979798316955566
step 20000, training accuracy 0.745
Accuracy validation: 0.7878788709640503
step 22000, training accuracy 0.79
Accuracy validation: 0.7919192314147949
step 24000, training accuracy 0.735
Accuracy validation: 0.8181818723678589
step 26000, training ac